In [1]:
import numpy as np
import pandas as pd
import os
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Reading Data

In [2]:
remove_chars = [chr(x) for x in range(31)]
add_spaces = ["[", "]", "(", ")", "`", ":", ".", ";", "{", "}"]
def segmentWords(s): 
    # Remove non-printable characters, change remove ending apostrophe or apostrophe s, as those
    # make the dataset more sparse, and dont provide additional value regarding sentiment in the bag of words model.
    # (Most of the time its just making an extra word for a proper noun)
    for char in remove_chars:
        s = s.replace(char,"")
    for char in add_spaces:
        s = s.replace(char,char + " ")
    words = [word if not word.endswith("'s") else word[:-2] for word in s.split()]
    return [word if not word.endswith("'") else word[:-1] for word in words]

    

def readFile(fileName):
    # Function for reading file
    # input: filename as string
    # output: contents of file as list containing single words
    contents = []
    f = open(fileName)
    for line in f:
        contents.append(line)
    f.close()
    result = segmentWords('\n'.join(contents))
    return result

#### Create a Dataframe containing the counts of each word in a file

In [3]:
d = []

for c in os.listdir("data_training"):
    directory = "data_training/" + c
    for file in os.listdir(directory):
        words = readFile(directory + "/" + file)
        e = {x:words.count(x) for x in words}
        e['__FileID__'] = file
        e['is_positive'] = 1 if c == 'pos' else 0
        e['num_words'] = sum([1 for x in words if len(x) > 1]) #Exclude punctuation and empty strings
        d.append(e)

Create a dataframe from d - make sure to fill all the nan values with zeros.

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html


In [4]:
df = pd.DataFrame(data=d,index=None)
if '' in df.columns:
    df.drop('', inplace=True, axis=1)
df.fillna(value=0,axis=0,inplace=True)
df = df.astype(np.int32, errors='ignore') # Become slightly more memory efficient, because my computer is a potato.
del d

In [5]:
df.head()

,!,"""",#,#05425,#1,#13,#2,#3,#5,#6,...,zuko,zukovsky,zulu,zundel,zurg,zweibel,zwick,zwigoff,zycie,|
0,0,24,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,42,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,19,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Split data into training and validation set 

* Sample 80% of your dataframe to be the training data

* Let the remaining 20% be the validation data (you can filter out the indicies of the original dataframe that weren't selected for the training data)

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [6]:
traindf, validatedf = train_test_split(df, test_size=0.2)

* Split the dataframe for both training and validation data into x and y dataframes - where y contains the labels and x contains the words

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html

In [7]:
traindf_x = traindf.drop('is_positive', axis=1)
traindf_x.drop('__FileID__', inplace=True, axis=1)
traindf_y = pd.DataFrame(traindf['is_positive'])

validatedf_x = validatedf.drop('is_positive', axis=1)
validatedf_x.drop('__FileID__', inplace=True, axis=1)
validatedf_y = pd.DataFrame(validatedf['is_positive'])

# Logistic Regression

#### Basic Logistic Regression
* Use sklearn's linear_model.LogisticRegression() to create your model.
* Fit the data and labels with your model.
* Score your model with the same data and labels.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

#### Changing Parameters

#### Feature Selection
* In the backward stepsize selection method, you can remove coefficients and the corresponding x columns, where the coefficient is more than a particular amount away from the mean - you can choose how far from the mean is reasonable.

References:

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.html#
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop.html
http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.where.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.std.html
https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.mean.html

How did you select which features to remove? Why did that reduce overfitting?

# Single Decision Tree

#### Basic Decision Tree

* Initialize your model as a decision tree with sklearn.
* Fit the data and labels to the model.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html


In [17]:
def most_important_features(dtc):
    lst = []
    for i in range(len(dtc.feature_importances_)):
        if dtc.feature_importances_[i] > 0:
            lst.append([dtc.feature_importances_[i], df.columns[i]])
    lst.sort(key=lambda x: x[0], reverse=True)
    return lst
dt_clf = DecisionTreeClassifier(criterion='entropy')
dt_clf.fit(traindf_x, traindf_y)
print("Unpruned decision tree score: ", dt_clf.score(validatedf_x, validatedf_y))

Unpruned decision tree score:  0.609375


#### Changing Parameters
* To test out which value is optimal for a particular parameter, you can either loop through various values or look into sklearn.model_selection.GridSearchCV

References:


http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html

In [16]:
parameters = {'criterion':['entropy'], 'max_depth':[x for x in range(3,13)]}
clf = GridSearchCV(DecisionTreeClassifier(), parameters)
clf.fit(X=traindf_x, y=traindf_y['is_positive'].values)
tree_model = clf.best_estimator_
print (clf.best_score_, clf.best_params_) 

0.64609375 {'criterion': 'entropy', 'max_depth': 5}


How did you choose which parameters to change and what value to give to them? Feel free to show a plot.

In [15]:
clf.score(X=validatedf_x, y=validatedf_y['is_positive'].values)

0.66249999999999998

Why is a single decision tree so prone to overfitting?

A single decision tree can train to fit the training data exactly, if it is not pruned. 

# Random Forest Classifier

#### Basic Random Forest

* Use sklearn's ensemble.RandomForestClassifier() to create your model.
* Fit the data and labels with your model.
* Score your model with the same data and labels.

References:

http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html


#### Changing Parameters

What parameters did you choose to change and why?

How does a random forest classifier prevent overfitting better than a single decision tree?